## Check DB Connection

In [1]:
import os
from sqlalchemy import create_engine
from dotenv import load_dotenv
from langchain_community.utilities import SQLDatabase

In [2]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

In [3]:
load_dotenv()

True

In [6]:
# Get the database credentials from environment variables
db_username = os.getenv('DB_USERNAME')
db_password = os.getenv('DB_PASSWORD')
db_host = os.getenv('DB_HOST')
db_port = os.getenv('DB_PORT')
db_name = os.getenv('DB_NAME')

In [7]:
db_url = f'postgresql://{db_username}:{db_password}@{db_host}:{db_port}/{db_name}'

db = SQLDatabase.from_uri(db_url)

## Create SQL Agent

In [8]:
from langchain_community.agent_toolkits import create_sql_agent 
from langchain_openai import ChatOpenAI

In [9]:
openai_api_key = os.getenv("OPENAI_API_KEY")
print(openai_api_key)

sk-proj-WbPv2R7YTidWgBSMvIjST3BlbkFJyjpl0kw0Ww7W96WFNx78


In [10]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, api_key=openai_api_key)

## Prompt Template engineering

In [11]:
context = db.get_context()
print(context["table_info"])


CREATE TABLE company (
	company_logo_url TEXT, 
	company_logo_text TEXT, 
	company_name TEXT, 
	relation_to_event TEXT, 
	event_url TEXT, 
	company_revenue BIGINT, 
	n_employees BIGINT, 
	company_phone TEXT, 
	company_founding_year BIGINT, 
	company_address TEXT, 
	company_industry TEXT, 
	company_overview TEXT, 
	homepage_url TEXT, 
	linkedin_company_url TEXT, 
	homepage_base_url TEXT, 
	company_logo_url_on_event_page TEXT, 
	company_logo_match_flag TEXT, 
	company_country TEXT
)

/*
3 rows from company table:
company_logo_url	company_logo_text	company_name	relation_to_event	event_url	company_revenue	n_employees	company_phone	company_founding_year	company_address	company_industry	company_overview	homepage_url	linkedin_company_url	homepage_base_url	company_logo_url_on_event_page	company_logo_match_flag	company_country
https://d1hbpr09pwz0sk.cloudfront.net/logo_url/100-women-in-finance-6a062f47	Women in Finance	100 Women In Finance	partner	https://apac.commoditytradingweek.com/	0	11	Un

In [ ]:
import ast
import re


def query_as_list(db, query):
    res = db.run(query)
    res = [el for sub in ast.literal_eval(res) for el in sub if el]
    res = [re.sub(r"\b\d+\b", "", string).strip() for string in res]
    return list(set(res))

In [23]:
system = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Try more advacned queries if the normal query does not result in any responses.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If you need to filter on a proper noun, you must ALWAYS first look up the filter value using the "search_proper_nouns" tool! 

You have access to the following tables: {table_names}

If the question does not seem related to the database, explain to the user why you can't find the answer.

Return the response in a nice format and visualize it if possible.

Use the following information about the tables to retrieve accurate answers.



CREATE TABLE company (
	company_logo_url TEXT, 
	company_logo_text TEXT, 
	company_name TEXT, 
	relation_to_event TEXT, 
	event_url TEXT, 
	company_revenue BIGINT, 
	n_employees BIGINT, 
	company_phone TEXT, 
	company_founding_year BIGINT, 
	company_address TEXT, 
	company_industry TEXT, 
	company_overview TEXT, 
	homepage_url TEXT, 
	linkedin_company_url TEXT, 
	homepage_base_url TEXT, 
	company_logo_url_on_event_page TEXT, 
	company_logo_match_flag TEXT, 
	company_country TEXT
)

/*
3 rows from company table:
company_logo_url	company_logo_text	company_name	relation_to_event	event_url	company_revenue	n_employees	company_phone	company_founding_year	company_address	company_industry	company_overview	homepage_url	linkedin_company_url	homepage_base_url	company_logo_url_on_event_page	company_logo_match_flag	company_country
https://d1hbpr09pwz0sk.cloudfront.net/logo_url/100-women-in-finance-6a062f47	Women in Finance	100 Women In Finance	partner	https://apac.commoditytradingweek.com/	0	11	Unknown	2001	None	Financial Services	100 Women in Finance strengthens the global finance industry by empowering women to achieve their pr	https://100women.org/events/	https://www.linkedin.com/company/100-women-in-finance/about	Unknown	https://apac.commoditytradingweek.com/wp-content/uploads/2022/03/100wif_web-1.png	yes	Unknown
https://media.licdn.com/dms/image/C4D0BAQHlTYAmrCwYOw/company-logo_200_200/0/1671349864369/bbgc_serv	BBGC	Bbgc	sponsor	https://apac.commoditytradingweek.com/	0	51	Unknown	0	None	It Services And It Consulting	Business Benefits Global Consulting (BBGC) is a multinational consultancy firm with offices in Middl	Unknown	https://it.linkedin.com/company/bbgcservices/about	Unknown	https://apac.commoditytradingweek.com/wp-content/uploads/2022/03/BBGC_WEB_logo-1.png	yes	Unknown
https://d1hbpr09pwz0sk.cloudfront.net/logo_url/hr-maritime-c13714f3	HR MARITIME	Hr Maritime	partner	https://apac.commoditytradingweek.com/	2	2	+41 22 732 57 00	2008	1-3 Rue De Chantepoulet, Geneva, Geneva 1201, CH	Maritime Transportation	HR Maritime is a Geneva based company providing services to the International Trading and Shipping i	http://www.hr-maritime.com	https://ch.linkedin.com/company/hr-maritime/about	Unknown	https://apac.commoditytradingweek.com/wp-content/uploads/2022/03/HR_logo-2.png	yes	CH
*/


CREATE TABLE event (
	event_logo_url TEXT, 
	event_name TEXT, 
	event_start_date TEXT, 
	event_end_date TEXT, 
	event_venue TEXT, 
	event_country TEXT, 
	event_description TEXT, 
	event_url TEXT, 
	event_industry TEXT
)

/*
3 rows from event table:
event_logo_url	event_name	event_start_date	event_end_date	event_venue	event_country	event_description	event_url	event_industry
http://expo.overseaseducation.sg/v.fastcdn.co/u/0e18fcd3/57786581-0-Untitled-.png	World University Expo @ SUNTEC	2024-07-06	2024-07-06	SUNTEC Convention Centre	None	Join our World University Expo this Saturday, 6 July 2024 12-6.30pm at SUNTEC whereby you can meet s	http://expo.overseaseducation.sg/	Education
https://ahiceconference.com/southeastasia/wp-content/uploads/sites/2/2024/05/AHICE-SEA-Logo.jpg	AHICE South East Asia	2025-02-25	2025-02-26	Pan Pacific Singapore	None	The Asian Hotel Industry Conference & Exhibition (AHICE) hosted by HM Magazine made its South East A	https://ahiceconference.com/southeastasia/	Education
https://apac.commoditytradingweek.com/wp-content/uploads/2024/02/cropped-ctw-apac-main-2.png	Commodity Trading Week APAC	2025-02-25	2025-02-26	Marina Bay Sands	None	Commodity Trading Week APAC is the premier event in the Asia Pacific region for the commodity indust	https://apac.commoditytradingweek.com/	Finance
*/


CREATE TABLE people (
	index BIGINT, 
	first_name TEXT, 
	middle_name TEXT, 
	last_name TEXT, 
	job_title TEXT, 
	person_city TEXT, 
	person_state TEXT, 
	person_country TEXT, 
	email_pattern TEXT, 
	homepage_base_url TEXT, 
	duration_in_current_job TEXT, 
	duration_in_current_company TEXT, 
	email_address TEXT, 
	email_valid BOOLEAN, 
	full_name TEXT, 
	years_in_current_job BIGINT
)

/*
3 rows from people table:
index	first_name	middle_name	last_name	job_title	person_city	person_state	person_country	email_pattern	homepage_base_url	duration_in_current_job	duration_in_current_company	email_address	email_valid	full_name	years_in_current_job
0	Cynthia	None	Battini	Indirect Buyer	None	None	France	None	ariane.group	None	None	None	False	Cynthia Battini	0
1	Alexander	None	McClure	Public Relations	Austin	TX	US	None	amazon.com	None	None	None	False	Alexander Mcclure	0
2	Nadir	None	Shaari	SEO Junior Account Manager	London	None	GB	[first].[last]	artefact.com	None	None	nadir.shaari@artefact.com	True	Nadir Shaari	0
*/

event and company tables can be merged using ‘event_url’ column.
company and people data can be merged using ‘homepage_base_url’ column.
Each event_url corresponds to a unique event, and each homepage_base_url can
be interpreted as a unique company.
 """

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("human", "{input}"), MessagesPlaceholder("agent_scratchpad")]
)
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=prompt,
    agent_type="tool-calling",
    verbose=True,
)

In [20]:
import pprint

chunks = []

async for chunk in agent.astream(
    {"input": "Find all sales people working in Singapore"}
):
    chunks.append(chunk)
    print("------")
    pprint.pprint(chunk, depth=5)



> Entering new SQL Agent Executor chain...
------
[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_mQkMHUsNs0comaEBrN4JEBoz', 'function': {'arguments': '{"query":"SELECT first_name, last_name, job_title, company_name, person_city, person_country FROM people_company WHERE job_title ILIKE \'%sales%\' AND person_country = \'Singapore\'"}', 'name': 'sql_db_query_checker'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0f03d4f0ee'}, id='run-ef866a58-9903-4ece-b03f-c1d948a00213', tool_calls=[{'name': 'sql_db_query_checker', 'args': {'query': "SELECT first_name, last_name, job_title, company_name, person_city, person_country FROM people_company WHERE job_title ILIKE '%sales%' AND person_country = 'Singapore'"}, 'id': 'call_mQkMHUsNs0comaEBrN4JEBoz', 'type': 'tool_call'}], tool_call_chunks=[{'name': 'sql_db_query_checker', 'args': '{"query":"SELECT first_name, la

In [13]:
agent.invoke({"input": "Identify companies that are attending finance related events."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_industry \nFROM company c \nJOIN event e ON c.relation_to_event = 'partner' OR c.relation_to_event = 'sponsor' \nWHERE e.event_industry = 'Finance';"}`


```sql
SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_industry 
FROM company c 
JOIN event e ON c.relation_to_event IN ('partner', 'sponsor') 
WHERE e.event_industry = 'Finance';
```
Invoking: `sql_db_query` with `{'query': "SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_industry \nFROM company c \nJOIN event e ON c.relation_to_event IN ('partner', 'sponsor') \nWHERE e.event_industry = 'Finance';"}`


[('Debeaurepairewines.Com', None, 'Commodity Trading Week APAC', 'Finance'), ('Aleios', 'Software Development', 'Accounting & Business Show Asia 2024', 'Finance'), ('Space Ambition', 'Space Research And Technology', '

{'input': 'Identify companies that are attending finance related events.',
 'output': 'Here are some companies that are attending finance-related events:\n\n| Company Name                          | Company Industry                                                                 | Event Name                          | Event Industry |\n|---------------------------------------|----------------------------------------------------------------------------------|-------------------------------------|----------------|\n| Debeaurepairewines.Com                | None                                                                             | Commodity Trading Week APAC         | Finance        |\n| Aleios                                | Software Development                                                              | Accounting & Business Show Asia 2024| Finance        |\n| Space Ambition                        | Space Research And Technology                                               

In [14]:
agent.invoke({"input": "Identify companies that are attending banking related events."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_industry \nFROM company c \nJOIN event e ON c.relation_to_event = 'partner' OR c.relation_to_event = 'sponsor' \nWHERE e.event_industry ILIKE '%banking%' OR e.event_description ILIKE '%banking%' \nORDER BY c.company_name;"}`


```sql
SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_industry 
FROM company c 
JOIN event e ON c.relation_to_event IN ('partner', 'sponsor') 
WHERE e.event_industry ILIKE '%banking%' OR e.event_description ILIKE '%banking%' 
ORDER BY c.company_name;
```
Invoking: `sql_db_query` with `{'query': "SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_industry \nFROM company c \nJOIN event e ON c.relation_to_event IN ('partner', 'sponsor') \nWHERE e.event_industry ILIKE '%banking%' OR e.event_description ILIKE '%banking%' \nORDER BY c.company_name;"}`



{'input': 'Identify companies that are attending banking related events.',
 'output': 'Here are some companies that are attending banking-related events:\n\n| Company Name                       | Company Industry                                   | Event Name                | Event Industry |\n|------------------------------------|---------------------------------------------------|---------------------------|----------------|\n| 100 Women In Finance               | Financial Services                                 | Seamless Asia 2025       | Other          |\n| Accelya Group                      | Financial Services                                 | Seamless Asia 2025       | Other          |\n| Aeropay                            | Financial Services, Mobile Payments, Fintech      | Seamless Asia 2025       | Other          |\n| Bloomberg                          | Financial Services                                 | Seamless Asia 2025       | Other          |\n| Federal Reserve Ban

In [15]:
agent.invoke({"input": "Identify companies that are attending Oil & Gas related events."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_description \nFROM company c \nJOIN event e ON c.relation_to_event = 'partner' OR c.relation_to_event = 'sponsor' \nWHERE e.event_industry ILIKE '%Oil & Gas%';"}`


```sql
SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_description 
FROM company c 
JOIN event e ON c.relation_to_event IN ('partner', 'sponsor') 
WHERE e.event_industry ILIKE '%Oil & Gas%';
```
Invoking: `sql_db_query` with `{'query': "SELECT DISTINCT c.company_name, c.company_industry, e.event_name, e.event_description \nFROM company c \nJOIN event e ON c.relation_to_event IN ('partner', 'sponsor') \nWHERE e.event_industry ILIKE '%Oil & Gas%';"}`


It appears that there are currently no companies listed as attending events related to the Oil & Gas industry in the database. If you have any other queries or need information on a di

{'input': 'Identify companies that are attending Oil & Gas related events.',
 'output': 'It appears that there are currently no companies listed as attending events related to the Oil & Gas industry in the database. If you have any other queries or need information on a different topic, feel free to ask!'}

In [16]:
agent.invoke({"input": "Find all sales people working in Singapore"})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT DISTINCT job_title FROM people;'}`
responded: It seems that the database does not have a specific column for job titles related to "sales" in the people table. However, I can search for people based on their job titles that might be related to sales, such as "Sales Representative," "Sales Manager," etc.

Let me first check the job titles available in the people table to see if there are any relevant titles related to sales. I'll proceed with that.

```sql
SELECT DISTINCT job_title FROM people;
```
Invoking: `sql_db_query` with `{'query': 'SELECT DISTINCT job_title FROM people;'}`


[('Property Consultant',), ('Area Director, Malaysia, Thailand and Brunei',), ('SVP Strategic Initatives - Europe CarbonClick',), ('Procurement',), ('customer manager',), ('Vice president sales',), ('Regional Manager @ Electrolux Professional',), ('Slocum Glider Commercial Sales Manager Teledyne Marine Vehi

{'input': 'Find all sales people working in Singapore',
 'output': 'Here are the sales professionals working in Singapore, along with their job titles:\n\n| First Name | Last Name | Job Title | City | Country |\n|------------|-----------|-----------|------|---------|\n| Koh        | Jing How  | Enterprise Sales SEA PayPal | Singapore | Singapore |\n| Eric       | Chan      | Sales Manager | Singapore | Singapore |\n| Poh        | -         | Land Transfer and Land Sales @ Singapore Land Authority (SLA) | Singapore | Singapore |\n| Cao        | Li        | Sales | Singapore | Singapore |\n| Melissa    | Wong      | Sales Administrative Executive | Singapore | Singapore |\n| Mitchell   | Chan      | Sales Engineer Splunk | Singapore | Singapore |\n| David      | Bochsler  | Vice President of Sales (Asia Pacific & Japan) KnowBe4 | Singapore | Singapore |\n| Patricia   | A.        | Sales Manager ASEAN Siemens | Singapore | Singapore |\n| Zhiming    | Fong      | Director Strategic Partner

In [17]:
agent.invoke({"input": "Find sales people working for over a year in Singapore."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT first_name, last_name, job_title, person_city, person_country, duration_in_current_job \nFROM people \nWHERE job_title ILIKE '%Sales%' \nAND person_country = 'Singapore' \nAND years_in_current_job > 1;"}`
responded: To find salespeople working for over a year in Singapore, I will first query the `people` table to filter for individuals with a job title related to sales and who have been in their current job for more than one year. 

Let's construct the query accordingly. 

1. Filter for job titles that include "Sales".
2. Filter for people located in Singapore.
3. Filter for those who have been in their current job for more than 1 year.

I'll proceed to check the query before executing it. 

Here's the query I will use:

```sql
SELECT first_name, last_name, job_title, person_city, person_country, duration_in_current_job 
FROM people 
WHERE job_title ILIKE '%Sales%' 
AND person_country

{'input': 'Find sales people working for over a year in Singapore.',
 'output': 'Here are the salespeople working in Singapore for over a year:\n\n| First Name | Last Name | Job Title                                                                 | City      | Country   | Years in Current Job |\n|------------|-----------|---------------------------------------------------------------------------|-----------|-----------|----------------------|\n| Koh        | Jing How  | Enterprise Sales SEA PayPal                                               | None      | Singapore | 6                    |\n| Patricia   | A.        | Sales Manager ASEAN Siemens                                               | Singapore | Singapore | 9                    |\n| Zhiming    | Fong      | Director Strategic Partnerships & Sales, APAC Amadeus                    | None      | Singapore | 2                    |\n| Annabel    | Chang     | Sales Development Manager (APJ) KnowBe4                                 

In [18]:
agent.invoke({"input": "Find the people working the longest in their current company."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT first_name, last_name, duration_in_current_company FROM people ORDER BY duration_in_current_company DESC LIMIT 5;'}`


```sql
SELECT first_name, last_name, duration_in_current_company FROM people ORDER BY duration_in_current_company DESC LIMIT 5;
```
Invoking: `sql_db_query` with `{'query': 'SELECT first_name, last_name, duration_in_current_company FROM people ORDER BY duration_in_current_company DESC LIMIT 5;'}`


[('Alexander', 'McClure', None), ('Nadir', 'Shaari', None), ('Daniela', 'Bonvino', None), ('Nikita', 'Kushwa', None), ('Cynthia', 'Battini', None)]It appears that the data for the duration of employment in the current company is not available (all values are `None`). Therefore, I cannot provide the names of people working the longest in their current company based on the available information. 

If you have any other questions or need information on a different topic, feel 

{'input': 'Find the people working the longest in their current company.',
 'output': 'It appears that the data for the duration of employment in the current company is not available (all values are `None`). Therefore, I cannot provide the names of people working the longest in their current company based on the available information. \n\nIf you have any other questions or need information on a different topic, feel free to ask!'}

In [19]:
agent.invoke({"input": "Find me the events happening in the next 6 months."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT event_name, event_start_date, event_end_date, event_venue, event_country FROM event WHERE event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '6 months' ORDER BY event_start_date;"}`


```sql
SELECT event_name, event_start_date, event_end_date, event_venue, event_country 
FROM event 
WHERE event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '6 months' 
ORDER BY event_start_date;
```
Invoking: `sql_db_query` with `{'query': "SELECT event_name, event_start_date, event_end_date, event_venue, event_country FROM event WHERE event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '6 months' ORDER BY event_start_date;"}`


Error: (psycopg2.errors.UndefinedFunction) operator does not exist: text >= date
LINE 1: ..., event_country FROM event WHERE event_start_date BETWEEN CU...
                                                             ^
HINT:  No operat

{'input': 'Find me the events happening in the next 6 months.',
 'output': "Here are the events happening in the next 6 months:\n\n| Event Name                                               | Start Date  | End Date    | Venue                                                        | Country               |\n|---------------------------------------------------------|-------------|-------------|--------------------------------------------------------------|-----------------------|\n| SG Pet Festival 2024                                    | 2024-08-02  | 2024-08-04  | Marina Bay Sands Singapore Convention Centre, Level 1, Hall A, B & C | None                  |\n| MBA and Master's Fair in Singapore                      | 2024-08-17  | 2024-08-17  | Carlton Hotel Singapore                                      | None                  |\n| The Business Show Asia                                   | 2024-08-28  | 2024-08-29  | Sands Expo & Convention Centre                               | None

In [20]:
agent.invoke({"input": "Find me the events happening in the next 12 months."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT event_name, event_start_date, event_end_date, event_venue, event_country FROM event WHERE event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '1 year' ORDER BY event_start_date;"}`


```sql
SELECT event_name, event_start_date, event_end_date, event_venue, event_country 
FROM event 
WHERE event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '1 year' 
ORDER BY event_start_date;
```
Invoking: `sql_db_query` with `{'query': "SELECT event_name, event_start_date, event_end_date, event_venue, event_country FROM event WHERE event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '1 year' ORDER BY event_start_date;"}`


Error: (psycopg2.errors.UndefinedFunction) operator does not exist: text >= date
LINE 1: ..., event_country FROM event WHERE event_start_date BETWEEN CU...
                                                             ^
HINT:  No operator mat

{'input': 'Find me the events happening in the next 12 months.',
 'output': "Here are the events happening in the next 12 months:\n\n| Event Name                                               | Start Date  | End Date    | Venue                                                        | Country               |\n|---------------------------------------------------------|-------------|-------------|--------------------------------------------------------------|-----------------------|\n| SG Pet Festival 2024                                    | 2024-08-02  | 2024-08-04  | Marina Bay Sands Singapore Convention Centre, Level 1, Hall A, B & C | None                  |\n| MBA and Master's Fair in Singapore                      | 2024-08-17  | 2024-08-17  | Carlton Hotel Singapore                                      | None                  |\n| ISPE Singapore Conference & Exhibition 2024             | 2024-08-28  | 2024-08-30  | Suntec Singapore                                            | None

In [24]:
agent.invoke({"input": "Find me the companies that are attending events in the next 3 months."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': "SELECT DISTINCT c.company_name, c.company_overview, e.event_name, e.event_start_date \nFROM company c \nJOIN event e ON c.event_url = e.event_url \nWHERE e.event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '3 months' \nORDER BY e.event_start_date;"}`


```sql
SELECT DISTINCT c.company_name, c.company_overview, e.event_name, e.event_start_date 
FROM company c 
JOIN event e ON c.event_url = e.event_url 
WHERE e.event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '3 months' 
ORDER BY e.event_start_date;
```
Invoking: `sql_db_query` with `{'query': "SELECT DISTINCT c.company_name, c.company_overview, e.event_name, e.event_start_date \nFROM company c \nJOIN event e ON c.event_url = e.event_url \nWHERE e.event_start_date BETWEEN CURRENT_DATE AND CURRENT_DATE + INTERVAL '3 months' \nORDER BY e.event_start_date;"}`


Error: (psycopg2.errors.UndefinedFunction) operator d

{'input': 'Find me the companies that are attending events in the next 3 months.',
 'output': "Here are some companies that are attending events in the next three months:\n\n| Company Name                          | Company Overview                                                                                                                                                                                                 | Event Name                          | Event Start Date |\n|---------------------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|-------------------------------------|------------------|\n| Alcatel Submarine Networks            | ASN Information: Alcatel Submarine Networks, part of Nokia, leads the industry in terms of transmission capacity and installed base with more than 750,000 km of o

In [25]:
agent.invoke({"input": "Find events that already over."})



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_query_checker` with `{'query': 'SELECT event_name, event_start_date, event_end_date FROM event WHERE event_end_date < CURRENT_DATE ORDER BY event_end_date DESC;'}`


```sql
SELECT event_name, event_start_date, event_end_date FROM event WHERE event_end_date < CURRENT_DATE ORDER BY event_end_date DESC;
```
Invoking: `sql_db_query` with `{'query': 'SELECT event_name, event_start_date, event_end_date FROM event WHERE event_end_date < CURRENT_DATE ORDER BY event_end_date DESC;'}`


Error: (psycopg2.errors.UndefinedFunction) operator does not exist: text < date
LINE 1: ...e, event_end_date FROM event WHERE event_end_date < CURRENT_...
                                                             ^
HINT:  No operator matches the given name and argument type(s). You might need to add explicit type casts.

[SQL: SELECT event_name, event_start_date, event_end_date FROM event WHERE event_end_date < CURRENT_DATE ORDER BY event_end_date

{'input': 'Find events that already over.',
 'output': 'Here are the events that have already concluded:\n\n| Event Name                                         | Start Date  | End Date    |\n|----------------------------------------------------|-------------|-------------|\n| AFA Creators Super Fest 2024                       | 2024-07-27  | 2024-07-28  |\n| 2024 IEEE International Conference on Multimedia and Expo | 2024-07-15  | 2024-07-19  |\n| ZAK Salaam India Expo                               | 2024-07-11  | 2024-07-14  |\n| World Food Fair                                     | 2024-07-04  | 2024-07-07  |\n| World University Expo @ SUNTEC                     | 2024-07-06  | 2024-07-06  |\n\nPlease note that the dates listed are in the future, and thus these events have not yet occurred. If you need information on past events, please let me know!'}